## Analisi distribuzione classi della cartella Spettrogrammi per riduzione della dimensionalità dei dati

In [13]:
import numpy as np
import utils 
import pandas as pd
import os
import matplotlib.pyplot as plt

## Tentativo di riduzione della dimensionalità dei dati con Structural Similarity Index (SSIM)

In [ ]:
def create_csv_for_folders(directory, k):
    data_dir = os.path.join(directory, 'dati ssim')
    os.makedirs(data_dir, exist_ok=True)

    for root, dirs, files in os.walk(directory):
        png_files = [file for file in files if file.endswith('.png')]
        if len(png_files) < k:
            continue
        df = pd.DataFrame([os.path.join(root, file) for file in png_files], columns=['FilePath'])
        folder_name = os.path.basename(root)
        df.to_csv(os.path.join(data_dir, f'df_paths_{folder_name}.csv'), index=False)


In [ ]:
create_csv_for_folders('Spettrogrammi/Target/', 8)

In [ ]:
create_csv_for_folders('Spettrogrammi/Non-Target/', 8)

In [116]:
def merge_csv_files(directory, output_file):
    files = os.listdir(directory)

    csv_files = [file for file in files if file.endswith('.csv')]

    df_list = []

    for csv_file in csv_files:
        df = pd.read_csv(os.path.join(directory, csv_file))
        df_list.append(df)

    merged_df = pd.concat(df_list, ignore_index=True)
    merged_df.to_csv(output_file, index=False)

In [180]:
merge_csv_files('Spettrogrammi/Target/dati ssim/SSIM', 'Spettrogrammi/Target/ssim_results.csv')

In [181]:
merge_csv_files('Spettrogrammi/Non-Target/dati ssim/SSIM', 'Spettrogrammi/Non-Target/ssim_results.csv')

In [182]:
df_non_target = pd.read_csv('Spettrogrammi/Non-Target/ssim_results.csv')
pd.set_option('display.float_format', '{:.3f}'.format)
df_non_target.describe()

,SSIM
count,675576.000
mean,0.903
std,0.048
min,0.386
25%,0.889
50%,0.915
75%,0.932
max,0.981


In [183]:
df_target = pd.read_csv('Spettrogrammi/Target/ssim_results.csv')
pd.set_option('display.float_format', '{:.3f}'.format)
df_target.describe()

,SSIM
count,4538823.000
mean,0.928
std,0.033
min,0.378
25%,0.919
50%,0.936
75%,0.948
max,0.969


## Eliminazione delle immagini con SSIM > 0.95
Le immagini con SSIM > 0.95 verranno eliminate solo dalle classi con più di 500 immagini. Questo per evitare di eliminare troppi dati dalle classi con meno immagini.

Per cercare di eliminare più campioni simili, si è deciso di verificare il numero di occorrenze di ogni path all'interno di ssim_results.csv, prendendo in esame solo la prima colonna (Image1). Questo ci permette di capire quante immagini sono simili tra loro e di conseguenza eliminare più campioni simili.

In [184]:
def filter_and_count(csv_path):
    df = pd.read_csv(csv_path)
    filtered_df = df[df['SSIM'] >= 0.95]
    count_df = filtered_df['Image1'].value_counts().reset_index()
    count_df.columns = ['Image1', 'Count']
    sorted_df = count_df.sort_values(by='Count', ascending=False)
    
    if sorted_df.empty:
        return None

    return sorted_df.iloc[0]['Image1']

In [185]:
def get_image2_paths(ssim_csv_path, image1_path):
    df = pd.read_csv(ssim_csv_path)
    similar_images_df = df[df['Image1'] == image1_path]
    similar_images_list = similar_images_df['Image2'].tolist()
    return similar_images_list

In [194]:
def remove_similar_images(ssim_csv_path, df_paths_csv_path, class_column):
    while True:
        isGood = False
        
        # Ottieni il percorso dell'immagine con il maggior numero di occorrenze in ssim_results.csv con SSIM >= 0.95
        image1_path = filter_and_count(ssim_csv_path)
        list_image2 = get_image2_paths(ssim_csv_path, image1_path)
        ssim_df = pd.read_csv(ssim_csv_path)
        ssim_df = ssim_df[ssim_df['Image1'] != image1_path]

        ssim_df.to_csv(ssim_csv_path, index=False)
        
        # Se filter_and_count restituisce None
        if image1_path is None:
            break
        
        path_parts = image1_path.split('/')
        if path_parts[2] in class_column:
            isGood = True
            
        if isGood:
            df_paths = pd.read_csv(df_paths_csv_path)
    
            for path in list_image2:
                df_paths = df_paths[df_paths['FilePath'] != path]
                
            df_paths.to_csv(df_paths_csv_path, index=False)
            
        else:
            continue

In [187]:
df_non_target = pd.read_csv('Spettrogrammi/Non-Target/df_paths.csv')
df_target = pd.read_csv('Spettrogrammi/Target/df_paths.csv')

df_non_target.to_csv('final_dataset/df_paths_non_target.csv', index=False)
df_target.to_csv('final_dataset/df_paths_target.csv', index=False)

In [188]:
def get_class_distribution(df, class_column):
    class_distribution = df[class_column].value_counts() > 500
    class_distribution = class_distribution.sort_values(ascending=True)
    return class_distribution

Rimozione da ssim_results.csv delle righe appartenti alle classi con meno di 500 immagini

In [192]:
def remove_rows_by_class(ssim_csv_path, class_column):
    ssim_df = pd.read_csv(ssim_csv_path)
    
    # Create a new column 'class' in the DataFrame by extracting the class from 'Image1'
    ssim_df['class'] = ssim_df['Image1'].apply(lambda x: x.split('/')[2])
    
    # Create a condition that checks if 'class' is in 'class_column'
    condition = ssim_df['class'].isin(class_column)
    
    # Use the condition to filter the DataFrame
    filtered_df = ssim_df[condition]
    
    # Save the filtered DataFrame to the original file
    filtered_df.to_csv(ssim_csv_path, index=False)

# Split Dataset in Train, validation e Test

### Estratti i numeri di audio per classe per non dividere lo stesso audio nei vari set ed eliminate le classi con meno di 8 audio distinti.

In [ ]:
df_target = pd.read_csv('Spettrogrammi/Target/df_paths.csv')
df_non_target = pd.read_csv('Spettrogrammi/Non-Target/df_paths.csv')

In [13]:
import csv
import os
from collections import defaultdict

def estrai_info_da_csv(input_csv,output_csv):
    classi = defaultdict(set)
    with open(input_csv, mode='r', newline='') as file:
        reader = csv.reader(file)
        first_row = next(reader)
        if first_row[0] == 'FilePath':
            first_row = next(reader)
        for row in reader:
            path = row[0]
            classe = path.split(os.sep)[1]
            audio = path.split(os.sep)[2].split('_resampled_')[0]
            classi[classe].add(audio)
    
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Classe', 'Numero Audio associato alla classe'])
        for classe, audio_set in classi.items():
            writer.writerow([classe, len(audio_set)])
    



In [18]:
estrai_info_da_csv('Spettrogrammi/Target/df_paths.csv', 'Spettrogrammi/Target/df_class_num.csv')
estrai_info_da_csv('Spettrogrammi/Non-Target/df_paths.csv', 'Spettrogrammi/Non-Target/df_class_num.csv')

In [32]:
def filter_csv(class_num_csv, csv_to_filter, output_di_file):
    df = pd.read_csv(class_num_csv)
    df_filtered = df[df['Numero Audio associato alla classe'] >= 8]
    
    df_paths = pd.read_csv(csv_to_filter)
    df_paths['Classe'] = df_paths['FilePath'].apply(lambda x: x.split('\\')[1])
    df_paths_filtered = df_paths[df_paths['Classe'].isin(df_filtered['Classe'])]
    df_paths_filtered = df_paths_filtered.drop(columns=['Classe'])
    df_paths_filtered.to_csv(output_di_file, index=False)

In [33]:
filter_csv('Spettrogrammi/Target/df_class_num.csv', 'Spettrogrammi/Target/df_paths.csv', 'Spettrogrammi/Target/df_paths_filtered.csv')
filter_csv('Spettrogrammi/Non-Target/df_class_num.csv', 'Spettrogrammi/Non-Target/df_paths.csv', 'Spettrogrammi/Non-Target/df_paths_filtered.csv')